In [85]:
import qube
import glob
import importlib
import inspect
import os
import ast
from collections import defaultdict
from sklearn.base import BaseEstimator
from git import Repo, Remote
from itertools import zip_longest

%qubed

In [241]:
def iter_funcs(class_node):
    decls = {}
    for n in ast.walk(class_node): 
        if isinstance(n, ast.FunctionDef) and n.name == '__init__':
            args = []
            for x in n.args.args:
                args.append(x.arg)
            for x, dx in zip_longest(reversed(args), reversed(n.args.defaults)):
                if x != 'self':
                    # print(f"{x} = {dx.value if dx else None}")
                    decls[x] = dx.value if dx and isinstance(dx, ast.Constant) else None
    # return dict(reversed(decls))
    return dict(reversed(list(decls.items())))

In [253]:
def flatten(decl_list):
    for d in decl_list:
        try:
            yield d.id
        except AttributeError:
            try:
                yield d.func.id
            except AttributeError:
                yield None

def is_decorated(decorator_list, decorator_name):
    for x in flatten(decorator_list):
        if x == decorator_name:
            return True
    return False

strats = defaultdict(list)
for file in glob.glob('**/*.py', recursive=True):
    name = os.path.splitext(os.path.basename(file))[0]
    # Ignore __ files
    if name.startswith("__"):
        continue

    with open(file, 'r') as f:
        src = f.read()

    try:
        class_node = ast.parse(src)
    except:
        continue

    nodes = [node for node in ast.walk(class_node) if isinstance(node, ast.ClassDef)]
    for n in nodes:
        if is_decorated(n.decorator_list, 'signal_generator'):
            strats[file].append((n.name, ast.get_docstring(n), iter_funcs(n)))

In [254]:
print(dict(strats))

{
    'glxy_s3/models/utils.py': [
        (
            'WalkForwardTest',
            'Walk forward test',
            {'estimator': None, 'train_period': 4, 'test_period': 1, 'units': 'W'}
        ),
        (
            'RollingVolatilityScaledSizer',
            None,
            {
                'estimator': None,
                'max_capital_in_risk': None,
                'vol_periods': None,
                'vol_factor': 1,
                'seasoned_timeframe': '1H',
                'train_period': 4,
                'test_period': 1,
                'units': 'W',
                'emergency_stop_loss': 0,
                'debug': False
            }
        )
    ],
    'glxy_s3/models/seasons/season0.py': [
        (
            'HappyHours',
            '"   Basic generator (no filtering)',
            {
                'capital': None,
                'threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursTopDogs',
            '"   Trade only those ones that showed good performance during training period',
            {
                'capital': None,
                'threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'passing_threshold': 0.1,
                'debug': False
            }
        ),
        (
            'HappyHoursTrendFollower',
            '"   Using simplest trend filter MA: trade longs only if price > MA',
            {
                'capital': None,
                'threshold': None,
                'period': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursRSI',
            '"   Try open postions only when RSI\n"    Here important note: we use RSI 
calculated on open prices (not closes):\n"    that is why we do not shift rsi\'s values   
\n"\n"    | 27-Oct-2022: RSI filter uses close prices',
            {
                'capital': None,
                'threshold': None,
                'period': None,
                'rsi_threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursIBS',
            '"   Filter on IBS values',
            {
                'capital': None,
                'threshold': None,
                'ibs_threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursFDI',
            '"   Fractal Dimensions Index as filter for market regime',
            {
                'capital': None,
                'threshold': None,
                'period': None,
                'fdi_min': None,
                'fdi_max': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'debug': False
            }
        ),
        (
            'HappyHoursRsiManhattan',
            '"   Mixed RSI filtered seasonality and trend filter',
            {
                'capital': None,
                'threshold': None,
                'period': None,
                'rsi_threshold': None,
                'use_abs_rets': True,
                'use_ranked_hours': False,
                'g_type': None,
                'g_tsi_atr_period': 100,
                'g_tsi_atr_smoother': 'kama',
                'g_tsi_scaling_factor': 12,
                'g_tii_H': 20,
                'g_tii_ti_period': 3,
                'g_tii_period': 3,
                'g_revert_entry_on_guard': False,
                'g_revert_entry_fraction': 0.5,
                'debug': False
            }
        ),
        (
            'HappyHoursManhattan',
            '"   Mixed seasonality and tren

In [ ]:
file = 'incubator/research/meanreverting/nimble/nimblers.py'
with open(file, 'r') as f:
    src = f.read()
    try:
        class_node = ast.parse(src)
    except:
        raise

    for n in ast.walk(class_node): 
        if isinstance(n, ast.FunctionDef) and n.name == '__init__':
            args = []
            for x in n.args.args:
                args.append(x.arg)

            for x, dx in zip_longest(reversed(args), reversed(n.args.defaults)):
                if x != 'self':
                    print(f"{x} = {dx.value if dx and isinstance(dx, ast.Constant) else None}")

In [258]:
from qube.booster.boosterai import Boo

# Git control

In [268]:
r = Repo('incubator')
r.head.commit.tree

<git.Tree "5e704e6adb002e5dd055c1bf08a03833162274ab">

In [269]:
r.index.diff()

[]

In [270]:
r.branches

[<git.Head "refs/heads/master">]